---sidebar_position: 1title: "RunnablePassthrough: Passing data through"keywords: [RunnablePassthrough, RunnableParallel, LCEL]---

# Passing data through | データの受け渡しRunnablePassthroughは、入力を変更せずにそのまま渡すことができるほか、追加のキーを付加して渡すことも可能です。これは一般に、RunnableParallelと組み合わせて使用され、マップ内で新しいキーにデータを割り当てる際に利用されます。> RunnablePassthrough allows to pass inputs unchanged or with the addition of extra keys. This typically is used in conjuction with RunnableParallel to assign data to a new key in the map.RunnablePassthrough()は単独で呼び出された場合、入力をそのまま受け取って通過させるだけです。> RunnablePassthrough() called on it's own, will simply take the input and pass it through.assignを用いて呼び出されるRunnablePassthrough（`RunnablePassthrough.assign(...)`）は、入力を受け取り、そしてassign関数に渡された追加の引数を加えます。> RunnablePassthrough called with assign (`RunnablePassthrough.assign(...)`) will take the input, and will add the extra arguments passed to the assign function.以下の例をご覧ください：> See the example below:

In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

上記の通り、`passed` キーは `RunnablePassthrough()` を用いて呼び出され、結果として `{'num': 1}` がそのまま渡されました。> As seen above, `passed` key was called with `RunnablePassthrough()` and so it simply passed on `{'num': 1}`.二行目では、数値を3倍にするラムダを用いて`RunnablePassthrough.assign`を使用しました。この場合、`extra`は`{'num': 1, 'mult': 3}`に設定され、これは`mult`キーを追加した元の値です。> In the second line, we used `RunnablePastshrough.assign` with a lambda that multiplies the numerical value by 3. In this cased, `extra` was set with `{'num': 1, 'mult': 3}` which is the original value with the `mult` key added.最後に、マップに`modified`という三つ目のキーを設定しました。これは、`num`に1を加える単一の値を設定するラムダを使用しており、結果として`modified`キーは値`2`になりました。> Finally, we also set a third key in the map with `modified` which uses a labmda to set a single value adding 1 to the num, which resulted in `modified` key with the value of `2`.

## Retrieval Example | 検索例以下の例では、RunnablePassthroughとRunnableMapを組み合わせて使用するユースケースを見ていきます。> In the example below, we see a use case where we use RunnablePassthrough along with RunnableMap.

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

ここでは、プロンプトへの入力が「context」と「question」というキーを持つマップであることを想定しています。ユーザーからの入力は質問だけです。従って、リトリバーを用いてコンテキストを取得し、「question」キーにユーザーの入力をパススルーする必要があります。このケースでは、RunnablePassthroughを使用して、ユーザーの質問をプロンプトとモデルに渡すことができます。> Here the input to prompt is expected to be a map with keys "context" and "question". The user input is just the question. So we need to get the context using our retriever and passthrough the user input under the "question" key. In this case, the RunnablePassthrough allows us to pass on the user's question to the prompt and model.